<a href="https://colab.research.google.com/github/VladKast/Tweat_dissaster_prediction/blob/main/Tweat_dissaster_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**task is to:**
classify if input tweet is about real disaster then predict 1 otherwise predict 0. 
Train (or fine-tune) a classification model and generate predictions for test samples.

dataset for the task:
https://drive.google.com/drive/folders/14duDq9LrB8-DReAG_rRhcsfqkUQUZoeR?usp=share_link


In [ ]:
# Upload file
from google.colab import files
files.upload()

In [2]:
# Import required libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from numpy.lib.shape_base import split
from sklearn.model_selection import train_test_split

In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [4]:
# Look at tetrain data
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Rundomize training data
train_data_shuffled = train_data.sample(frac=1, random_state=42)

In [6]:
# Look at test data
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# Number of samples in each category
train_data_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# Data into training and validation sets
training_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data_shuffled["text"].to_numpy(),
                                                                            train_data_shuffled["target"].to_numpy(),
                                                                            test_size=0.15, # 15% training data for validation
                                                                            random_state=42)

In [9]:
print("number of training senteses:")
len(training_sentences)

number of training senteses:


6471

In [10]:
print("number of training labels:")
len(train_labels)

number of training labels:


6471

In [11]:
print("number of validation sentenses:")
len(val_sentences)

number of validation sentenses:


1142

In [12]:
print("number of validation labels:")
len(val_labels)

number of validation labels:


1142

# **Converting text into numbers** 

In [13]:
text_vectorizer = TextVectorization(max_tokens=None, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace",
                                    ngrams=None, 
                                    output_mode="int", 
                                    output_sequence_length=None) 

In [14]:
# Find the average number of words in the training tweets
round(sum([len(i.split())for i in training_sentences])/ len(training_sentences))

15

In [15]:
# Vectorization variables
max_vocab_length = 10000 
max_length = 15 
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                     output_sequence_length=max_length)

In [16]:
# Fit  the text vectorizer to the training text
text_vectorizer.adapt(training_sentences)

In [17]:
embedding = layers.Embedding(input_dim= max_vocab_length, 
                             output_dim=128, 
                             embeddings_initializer="uniform",
                             input_length=max_length 
                             )

In [18]:
print(embedding)

In [19]:
# Get a random sentence from the training set
random_sentence = random.choice(training_sentences)
print(f"Original text:{random_sentence}\
      \nEmbedded version:")

# Embed the random sentence
embed = embedding(text_vectorizer([random_sentence]))
embed

Original text:@DanRyckert @drewscanlon He's blazing through this game with the best stealth skills yet. Nothing beats the silenced M4.      
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01018001, -0.03430416,  0.04204634, ...,  0.03500697,
          0.02454839, -0.00460516],
        [ 0.01018001, -0.03430416,  0.04204634, ...,  0.03500697,
          0.02454839, -0.00460516],
        [-0.04811785, -0.02151507,  0.04506933, ..., -0.0371575 ,
          0.0259021 ,  0.04607365],
        ...,
        [ 0.03358493,  0.04845811, -0.02772012, ..., -0.04467652,
          0.02594345, -0.01723857],
        [-0.00290066,  0.03793813, -0.0425073 , ..., -0.03618175,
          0.02607981, -0.04229559],
        [ 0.02483175,  0.02014403,  0.04719985, ...,  0.03060416,
          0.02969792,  0.04905349]]], dtype=float32)>

**Creating a neural network**

In [20]:
# Set random seed
tf.random.set_seed(42)
# Create model
model = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer, embedding,
    layers.GlobalAveragePooling1D(),
    layers.Dense(10, activation = "sigmoid"),
   # layers.Dense(2, activation = "sigmoid"),
    layers.Dense(1, activation = "sigmoid")
])
# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(lr=0.001),
                metrics=["mae"])
# Fit the model
model.fit(training_sentences,train_labels,validation_data=(val_sentences, val_labels), epochs=6)

Epoch 1/6


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


203/203 [==============================] - 3s 13ms/step - loss: 0.7092 - mae: 0.4873 - val_loss: 0.6025 - val_mae: 0.4400
Epoch 2/6
203/203 [==============================] - 2s 12ms/step - loss: 0.5204 - mae: 0.3879 - val_loss: 0.5025 - val_mae: 0.3626
Epoch 3/6
203/203 [==============================] - 2s 11ms/step - loss: 0.4070 - mae: 0.2997 - val_loss: 0.4679 - val_mae: 0.3118
Epoch 4/6
203/203 [==============================] - 2s 12ms/step - loss: 0.3267 - mae: 0.2356 - val_loss: 0.4580 - val_mae: 0.2864
Epoch 5/6
203/203 [==============================] - 2s 12ms/step - loss: 0.2648 - mae: 0.1886 - val_loss: 0.4715 - val_mae: 0.2752
Epoch 6/6
203/203 [==============================] - 2s 12ms/step - loss: 0.2162 - mae: 0.1521 - val_loss: 0.4853 - val_mae: 0.2607


In [21]:
# Check the results
model.evaluate(val_sentences, val_labels)

36/36 [==============================] - 0s 2ms/step - loss: 0.4853 - mae: 0.2607


[0.4853192865848541, 0.26073917746543884]

In [22]:
# Make predictions and evaluation
model_pred_probs = model.predict(val_sentences)
model_pred_probs.shape

36/36 [==============================] - 0s 2ms/step


(1142, 1)

In [23]:
# look at the first "x" predictions
model_pred_probs[:15]

array([[0.37839463],
       [0.8371974 ],
       [0.9818521 ],
       [0.10045797],
       [0.10472175],
       [0.9660205 ],
       [0.96862984],
       [0.98151785],
       [0.9649868 ],
       [0.14246903],
       [0.14199427],
       [0.64296347],
       [0.09541085],
       [0.2240414 ],
       [0.04729692]], dtype=float32)

In [24]:
# Convert model prediction probabilities to 1 and 0 
model_predictions = tf.squeeze(tf.round(model_pred_probs))
model_predictions[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

### **Use model to make predictions**

In [25]:
# Make predictions 
prediction_probs = model.predict(test_data['text'].to_numpy())
# Predictions to labels
predictions = tf.squeeze(tf.round(prediction_probs))
predictions = tf.cast(predictions, dtype=tf.int32)

102/102 [==============================] - 0s 2ms/step


In [26]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [27]:
# Show results
show_results = pd.DataFrame({"id": test_data['id'].values,
                          'goal': (predictions.numpy())})

In [28]:
show_results[:20]

,id,goal
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [29]:
# show predictions
predictions[:20]

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
      dtype=int32)>